In [1]:
# import all necessary packages
import pandas as pd
import numpy as np
import json
import datetime
import warnings
from pandas.core.common import SettingWithCopyWarning

#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
#import seaborn as sns
#import matplotlib as mlp
import matplotlib.pyplot as plt
#from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.svm import SVR
import math
from sklearn import datasets
from sklearn.metrics import mean_squared_error

In [2]:
# import datasets
df_h_comar = pd.read_csv("data/datasets/df_h_comar.csv")
df_h_comar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227660 entries, 0 to 227659
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date_start             227660 non-null  object 
 1   start_stamp            227660 non-null  int64  
 2   Pickup_Community_Area  227660 non-null  float64
 3   numOfTaxis_area        227660 non-null  int64  
 4   start_time_hourly      227660 non-null  int64  
 5   dayOfWeek              227660 non-null  object 
 6   isHoliday              227660 non-null  bool   
 7   isRushhour             227660 non-null  bool   
 8   start_time_day         227660 non-null  int64  
 9   start_time_week        227660 non-null  int64  
 10  start_time_month       227660 non-null  int64  
 11  pressure               227660 non-null  float64
 12  humidity               227660 non-null  float64
 13  wind_direction         227660 non-null  float64
 14  wind_speed             227660 non-nu

In [3]:
df_h_comar.drop(columns = ['date_start'])

,start_stamp,Pickup_Community_Area,numOfTaxis_area,start_time_hourly,dayOfWeek,isHoliday,isRushhour,start_time_day,start_time_week,start_time_month,pressure,humidity,wind_direction,wind_speed,description,season,temperature_celsius
0,0,1.0,4,0,Tuesday,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,winter,-0.19
1,0,2.0,1,0,Tuesday,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,winter,-0.19
2,0,3.0,43,0,Tuesday,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,winter,-0.19
3,0,4.0,13,0,Tuesday,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,winter,-0.19
4,0,5.0,17,0,Tuesday,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,winter,-0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227655,23,59.0,1,23,Tuesday,False,False,31,1,12,1023.0,89.0,217.0,1.0,mist,winter,-11.00
227656,23,60.0,3,23,Tuesday,False,False,31,1,12,1023.0,89.0,217.0,1.0,mist,winter,-11.00
227657,23,61.0,1,23,Tuesday,False,False,31,1,12,1023.0,89.0,217.0,1.0,mist,winter,-11.00
227658,23,76.0,3,23,Tuesday,False,False,31,1,12,1023.0,89.0,217.0,1.0,mist,winter,-11.00


In [4]:
# defining categorical and numeric features of dfPhiladelphia

categoric = ['start_stamp', 'Pickup_Community_Area','dayOfWeek','start_time_month','start_time_day','start_time_week',
             'isHoliday','description','isRushhour', 'season']
numeric = ['temperature_celsius','wind_speed','wind_direction','humidity','pressure']


In [5]:
# function for normalize numeric and encode categorical features and for create pipeline

def pipeline_for_prediction(categoric, numeric, model):
    
    numeric_transformer = Pipeline(steps=[("standard_scaler", StandardScaler())])
    categorical_transformer = Pipeline(
        steps=[("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"))]
    )
    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical scaler", numeric_transformer, numeric),
            ("one hot encoder", categorical_transformer, categoric),
        ]
    )
    pipeline = Pipeline(
        steps=[("preprocessor", preprocessor), ("model", model)]
    )
    return pipeline

In [6]:
# function for getting different scores for a model

def get_prediction_scores(y_true, y_predicted):
    print("MODEL SCORES:")
    print(f"MAE: {metrics.mean_absolute_error(y_true, y_predicted): .3f}")
    print(f"MSE: {metrics.mean_squared_error(y_true, y_predicted): .3f}")
    print(f"RMSE: {math.sqrt(metrics.mean_squared_error(y_true, y_predicted)): .3f}")
    print(f"Accuracy:", round((1-(metrics.mean_absolute_error(y_true, y_predicted)/df_h_comar["numOfTaxis_area"].mean()))*100,2), "%")
    print(f"R2: {100 * metrics.r2_score(y_true, y_predicted): .3f} %")
    print(f"Max Residual Error: {metrics.max_error(y_true, y_predicted): .3f}")

In [7]:
# function for creating pipeline and fitting model (created by the pipeline), predict and printing scores

def pipeline_fit_predict(reg, categoric, numeric, x_train, y_train, x_val, y_val):
    pipeline = pipeline_for_prediction(categoric, numeric, reg)
    pipeline.fit(x_train, y_train)
    y_predict = pipeline.predict(x_val)
    get_prediction_scores(y_val, y_predict)

In [8]:
from sklearn.model_selection import train_test_split

#split the data set in 70% training set and 30% testing set
#x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.3,random_state=42)
x_train, x_test, y_train, y_test = train_test_split(df_h_comar.drop('numOfTaxis_area', axis=1)
                                                    , df_h_comar['numOfTaxis_area'], 
                                                    test_size=0.3,random_state=42)

# save the combination of training and validation set in extra variables
x_train_val = x_train
y_train_val = y_train

#split the training data set in 70% training set and 20% validation set to achieve a 50-20-30 split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=(0.2/0.7), random_state = 42)

## Linear Regression (as Benchmark)

In [9]:
# creating the regression model

lin_reg = LinearRegression()

In [10]:
pipeline = pipeline_fit_predict(lin_reg, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

MODEL SCORES:
MAE:  30.758
MSE:  3904.656
RMSE:  62.487
Accuracy: 21.97 %
R2:  62.137 %
Max Residual Error:  1339.486


## Support Vector Machine

### Checking for the best kernel

#### Kernel: linear

In [24]:
svr_lin = SVR(kernel = 'linear',verbose = 10,cache_size=15000, max_iter=50000)

In [12]:
pipeline = pipeline_fit_predict(svr_lin, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -3363473.268517, rho = -11.201561
nSV = 95981, nBSV = 95028


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  23.337
MSE:  4498.339
RMSE:  67.070
Accuracy: 40.8 %
R2:  56.381 %
Max Residual Error:  1446.159


#### Kernel: poly degree 2

In [18]:
svr_poly2 = SVR(kernel = 'poly',degree = 2, verbose = 10,cache_size=15000, max_iter=50000)

In [19]:
pipeline = pipeline_fit_predict(svr_poly, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -4735355.356241, rho = -6.414818
nSV = 99966, nBSV = 99966


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  26.649
MSE:  7286.713
RMSE:  85.362
Accuracy: 32.39 %
R2:  29.343 %
Max Residual Error:  1635.635


#### Kernel: poly degree 3

In [13]:
svr_poly3 = SVR(kernel = 'poly',verbose = 10,cache_size=15000, max_iter=50000)

In [14]:
pipeline = pipeline_fit_predict(svr_poly3, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -5031586.430808, rho = 4.514757
nSV = 99968, nBSV = 99968


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  35.897
MSE:  8870.264
RMSE:  94.182
Accuracy: 8.93 %
R2:  13.987 %
Max Residual Error:  1689.392


#### Kernel: rbf

In [16]:
svr_rbf = SVR(kernel = 'rbf',verbose = 10,cache_size=15000, max_iter=50000)

In [17]:
pipeline = pipeline_fit_predict(svr_rbf, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -4792854.643942, rho = -6.669636
nSV = 99984, nBSV = 99984


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  26.690
MSE:  7261.736
RMSE:  85.216
Accuracy: 32.29 %
R2:  29.585 %
Max Residual Error:  1634.289


In [20]:
# function for finding the best hyperparameter by using RandomizedSearchCV and RepeatedStratifiedKFold
"""parameter:
   - pipeline: used pipeline for grid search (the pipeline contains the model)
   - x_val: data set (features) used for grid search
   - y_val: data set (target value) used for grid search
   - model_par: parameters for which the grid search is done
   - score: used score measure 
   - n_iter: how often grid search will be done
   - n_repeats: how often the data set is randomly splitted (by using the same random hyperparameter) in n_splits
   - n_splits: number of splits in RepeatedStratifiedKFold
   - verbose: getting information during the grid search
"""


from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold

def find_best_hyperparameters(pipeline, x_val, y_val, model_par, score, n_iter = 50,  
                                   n_repeats=3, n_splits=5, n_jobs=1, verbose=True):
    
    print(f"Running grid search for the model based on {score}")
    grid_pipeline = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=model_par,
        n_jobs=n_jobs,
        n_iter=n_iter,
        cv=RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42),
        scoring=score,
        random_state=42,
        verbose=verbose,
    )
    grid_pipeline.fit(x_val, y_val)
    print(f"Best {score} Score was: {grid_pipeline.best_score_}")
    print("The best hyper parameters for the model are:")
    print(grid_pipeline.best_params_)

In [34]:
# creating the ranges for model parameter to use in find_best_hyperparameters

from scipy.stats import loguniform

model_para = {'model__C':loguniform(1e-1, 1e2),        
                'model__epsilon':loguniform(1e-1, 1e2)
             }               

In [35]:
svr_lin = SVR(kernel = 'linear',cache_size=15000, max_iter=50000)

In [36]:
pipeline = pipeline_for_prediction(categoric, numeric, svr_lin)

In [ ]:
find_best_hyperparameters(pipeline, x_val, y_val, model_para, score = 'neg_mean_squared_error', verbose=10)

Running grid search for the model based on neg_mean_squared_error
Fitting 15 folds for each of 50 candidates, totalling 750 fits
[CV 1/15; 1/50] START model__C=1.3292918943162162, model__epsilon=71.14476009343417


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/15; 1/50] END model__C=1.3292918943162162, model__epsilon=71.14476009343417;, score=-5269.594 total time=  10.5s
[CV 2/15; 1/50] START model__C=1.3292918943162162, model__epsilon=71.14476009343417
[CV 2/15; 1/50] END model__C=1.3292918943162162, model__epsilon=71.14476009343417;, score=-5384.253 total time=  10.5s
[CV 3/15; 1/50] START model__C=1.3292918943162162, model__epsilon=71.14476009343417
[CV 3/15; 1/50] END model__C=1.3292918943162162, model__epsilon=71.14476009343417;, score=-5483.595 total time=  10.6s
[CV 4/15; 1/50] START model__C=1.3292918943162162, model__epsilon=71.14476009343417
[CV 4/15; 1/50] END model__C=1.3292918943162162, model__epsilon=71.14476009343417;, score=-5109.788 total time=  10.6s
[CV 5/15; 1/50] START model__C=1.3292918943162162, model__epsilon=71.14476009343417
[CV 5/15; 1/50] END model__C=1.3292918943162162, model__epsilon=71.14476009343417;, score=-5464.847 total time=  10.4s
[CV 6/15; 1/50] START model__C=1.3292918943162162, model__epsilon=71.

/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 1/15; 2/50] END model__C=15.702970884055382, model__epsilon=6.251373574521747;, score=-4337.395 total time= 1.0min
[CV 2/15; 2/50] START model__C=15.702970884055382, model__epsilon=6.251373574521747


...

[CV 11/15; 8/50] END model__C=0.3511356313970406, model__epsilon=0.3549878832196502;, score=-5433.045 total time= 1.5min
[CV 12/15; 8/50] START model__C=0.3511356313970406, model__epsilon=0.3549878832196502


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 12/15; 8/50] END model__C=0.3511356313970406, model__epsilon=0.3549878832196502;, score=-5690.017 total time= 1.5min
[CV 13/15; 8/50] START model__C=0.3511356313970406, model__epsilon=0.3549878832196502


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 13/15; 8/50] END model__C=0.3511356313970406, model__epsilon=0.3549878832196502;, score=-5847.952 total time= 1.5min
[CV 14/15; 8/50] START model__C=0.3511356313970406, model__epsilon=0.3549878832196502
